## Imports

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px # this is another plotting library for interactive plot

from sklearn.model_selection import train_test_split
from sklearn import metrics, manifold # we will use the metrics and manifold learning modules from scikit-learn
from pathlib import Path # to interact with file paths
from PIL import Image # to interact with images
from tqdm import tqdm # progress bar
from pprint import pprint # pretty print (useful for a more readable print of objects like lists or dictionaries)

import torch
import torch.nn as nn
import torchvision
from torchvision.io import read_image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import cv2 as cv

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

## Load Pretrained Net and create Detector 

In [4]:
# TESTING, DONT RUN THIS CELL
# Model
from helper_functions import *
# model = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained=True) #faster but less accurate
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True) 
# model = torch.hub.load('ultralytics/yolov5', 'yolov5n6', pretrained=True) 
# model = torch.hub.load('ultralytics/yolov3', 'yolov3') #bad 
imgs = []

for i in range(1000):

    img = cv.imread(f'test_imgs/img_{i+1}.png')

    results = model(img)

    # Results
    results.print()
    results.save() # or .show()
    results.pandas().xyxy[0]  # img1 predictions (pandas)
    try:
        results.xyxy[0]  # img1 predictions (tensor)
        bb = results.xyxy[0].cpu().numpy()
        bb = bb[0,:4]
        print(bb)
        draw_bounding_box(img, bb)
        cv.imshow('img', img)
        cv.waitKey(1)
    except:
        print('no predictions')

    #save img 
    cv.imwrite(f'sign_imgs/detection_results/img_{i+1}.png', img)


cv.destroyAllWindows()
# results.pandas().xyxy[0]  # img1 predictions (pandas)

Using cache found in /home/irong/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-2-17 torch 1.10.0+cu113 CUDA:0 (NVIDIA GeForce GTX 950M, 2004MiB)

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
image 1/1: 480x640 (no detections)
Speed: 2.5ms pre-process, 40.5ms inference, 0.8ms NMS per image at shape (1, 3, 480, 640)
Saved 1 image to runs/detect/exp2053
image 1/1: 480x640 (no detections)
Speed: 2.0ms pre-process, 40.5ms inference, 1.9ms NMS per image at shape (1, 3, 480, 640)
Saved 1 image to runs/detect/exp2054
image 1/1: 480x640 (no detections)
Speed: 5.0ms pre-process, 40.3ms inference, 1.3ms NMS per image at shape (1, 3, 480, 640)
Saved 1 image to runs/detect/exp2055


no predictions
no predictions
no predictions

image 1/1: 480x640 (no detections)
Speed: 2.2ms pre-process, 40.2ms inference, 2.0ms NMS per image at shape (1, 3, 480, 640)
Saved 1 image to runs/detect/exp2056
image 1/1: 480x640 (no detections)
Speed: 2.0ms pre-process, 40.3ms inference, 2.0ms NMS per image at shape (1, 3, 480, 640)



no predictions


Saved 1 image to runs/detect/exp2057
image 1/1: 480x640 (no detections)
Speed: 2.3ms pre-process, 40.4ms inference, 1.2ms NMS per image at shape (1, 3, 480, 640)
Saved 1 image to runs/detect/exp2058
image 1/1: 480x640 (no detections)
Speed: 2.1ms pre-process, 44.4ms inference, 4.2ms NMS per image at shape (1, 3, 480, 640)


...

Saved 1 image to runs/detect/exp2287
image 1/1: 480x640 1 umbrella
Speed: 2.2ms pre-process, 40.3ms inference, 2.6ms NMS per image at shape (1, 3, 480, 640)
Saved 1 image to runs/detect/exp2288


no predictions
[     97.045       246.7      545.14      476.72]


image 1/1: 480x640 1 umbrella
Speed: 2.3ms pre-process, 60.8ms inference, 12.0ms NMS per image at shape (1, 3, 480, 640)
Saved 1 image to runs/detect/exp2289


[     108.01      417.14      535.92      478.68]


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
# Model
# model = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained=True) #faster but less accurate
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True) 
# model = torch.hub.load('ultralytics/yolov5', 'yolov5n6', pretrained=True) 
# model = torch.hub.load('ultralytics/yolov3', 'yolov3') #bad 
model.to(device)

# Analyze network

# print(model)

# for param_name, param in model.named_parameters():
#     print(param_name)

# for i, (k, v) in enumerate(model.named_parameters()):
#     print(f'{i} - {k}')

#https://github.com/ultralytics/yolov5/issues/1314

#backnbone is layers 0->9

backbone_layers = [f'model.{x}' for x in range(10)]

backbone = nn.Sequential(
    model.model.model.model[0],
    model.model.model.model[1],
    model.model.model.model[2],
    model.model.model.model[3],
    model.model.model.model[4],
    model.model.model.model[5],
    model.model.model.model[6],
    model.model.model.model[7],
    model.model.model.model[8],
    model.model.model.model[9],
    model.model.model.model[10],
    )

# print(backbone)

class FeatureExtractor(nn.Module):
    def __init__(self, backbone): #(default for 640x320)
        super().__init__()

        ## Pretrained layers
        self.pretrained = backbone

        ### Flatten layer
        self.flatten = nn.Flatten(start_dim=1)

    def forward(self, x):
        # Apply convolutions
        x = self.pretrained(x)
        # Flatten
        x = self.flatten(x)
        return x

class Detector(nn.Module):
    def __init__(self, add_inputs=4, regr_out=22, class_out=13, features=76800): #(default for 640x320)
        super().__init__()
        ### Linear sections
        self.lin = nn.Sequential(
            # First linear layer
            nn.Linear(in_features=features+add_inputs, out_features=512),
            nn.ReLU(True),
            nn.Linear(in_features=512, out_features=regr_out+class_out),
        )
        # #regression output
        # self.lin_regr = nn.Sequential(
        #     # First linear layer
        #     nn.Linear(in_features=1024, out_features=512),
        #     nn.ReLU(True),
        #     # nn.Dropout(p=0.5),
        #     # Second linear
        #     nn.Linear(in_features=512, out_features=regr_out)
        # )
        # # classification output
        # self.lin_class = nn.Sequential(
        #     # First linear layer
        #     nn.Linear(in_features=1024, out_features=512),
        #     nn.ReLU(True),
        #     # nn.Dropout(p=0.5),
        #     # Second linear
        #     nn.Linear(in_features=512, out_features=class_out)
        # )
        
    def forward(self, x):
        x = self.lin(x)
        return x

feature_extractor=FeatureExtractor(backbone)
#define detector
detector = Detector(add_inputs=4, regr_out=22-16+2, class_out=13, features=20480)

#freeeze backbone
for param in feature_extractor.pretrained.parameters():
    param.requires_grad = False

feature_extractor.to(device)
detector.to(device)

# #check
# for param_name, param in detector.named_parameters():
#     print('%s \t- requires_grad=%s' % (param_name, param.requires_grad))

In [ ]:
# test backbone
#show the image with opencv
img = cv.imread('tests/test_img.jpg')
#resize to 480 x 640
img = cv.resize(img, (320, 240))
#convert to tensor
img = torch.from_numpy(img).float().permute(2, 0, 1)
#add dimension
img = img.unsqueeze(0).to(device)
print(img.shape)

detector.eval()

# Inference
with torch.no_grad():
    data = torch.zeros(1, 4).to(device)
    feat = feature_extractor(img)
    input = torch.cat((feat, data), dim=1)
    print(input.shape)
    out = detector(input) 
    print(out.shape) # (320, 240)->torch.Size([1, 20480])
                    # (640, 480)->torch.Size([1, 76800])





## Loading images and Labels

In [ ]:
#dataset
class CsvDataset(Dataset):
    def __init__(self, folder, transform=None, in_ram=False):
        self.transform = transform
        self.in_ram = in_ram
        self.folder = folder
        self.data = []
        class_labels = []
        with open(folder+'/classification_labels.csv', 'r') as f:
            lines = f.read().split('\n')
            lines = lines[0:-1] #remove footer
            # Get x and y values from each line and append to self.data
            labels = []
            for i in tqdm(range(len(lines))):
                line = lines[i]
                sample = line.split(',')
                #convert to float
                label = np.array([float(s) for s in sample])
                #convert to tensor
                label = torch.from_numpy(label).float()
                # img = img.unsqueeze(0)
                class_labels.append(label)

        input_data = []
        with open(folder+'/input_data.csv', 'r') as f:
            lines = f.read().split('\n')
            lines = lines[0:-1] #remove footer
            # Get x and y values from each line and append to self.data
            labels = []
            for i in tqdm(range(len(lines))):
                line = lines[i]
                sample = line.split(',')
                #convert to float
                label = np.array([float(s) for s in sample])
                #convert to tensor
                label = torch.from_numpy(label).float()
                # img = img.unsqueeze(0)
                input_data.append(label)

        #load labels
        with open(folder+'/regression_labels.csv', 'r') as f:
            lines = f.read().split('\n')
            lines = lines[0:-1] #remove footer
            # Get x and y values from each line and append to self.data
            labels = []
            for i in tqdm(range(len(lines))):
                line = lines[i]
                sample = line.split(',')
                #convert to float
                label = np.array([float(s) for s in sample])
                #convert to tensor
                label = torch.from_numpy(label).float()
                #load img
                # img = cv.imread(folder+f'/img_{i+1}.png')
                # img = cv.resize(img, (320, 240))
                if self.in_ram:
                    img = cv.imread(folder+f'/img_{i+1}.png')
                    img = cv.resize(img, (320, 240))
                    img = torch.from_numpy(img).float().permute(2, 0, 1)
                    self.data.append((img, input_data[i], label, class_labels[i]))  
                else:
                    self.data.append((input_data[i], label, class_labels[i])) #no image
                    #save img to disk
                    # cv.imwrite(folder+f'/img_{i+1}.png', img)

    def __len__(self):
        # The length of the dataset is simply the length of the self.data list
        return len(self.data)

    def __getitem__(self, idx):
        # Our sample is the element idx of the list self.data
        sample = self.data[idx]
        if not self.in_ram:
            # img = read_image(read_image(img_path))
            img = cv.imread(self.folder+f'/img_{idx+1}.png')
            img = cv.resize(img, (320, 240))
            img = torch.from_numpy(img).float().permute(2, 0, 1)
            sample = (img, sample[0], sample[1], sample[2])
        if self.transform:
            sample = self.transform(sample)
        return sample

#create dataset
train_dataset = CsvDataset(folder='training_imgs')

train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)

In [ ]:
#test dataloader
sample = next(iter(train_dataloader))
print(sample[0].shape)
print(sample[1].shape)
print(sample[2].shape)
print(sample[3].shape)

## Training

In [ ]:
# Training function
def train_epoch(ext, det, dataloader, class_loss_fn, regr_loss_fn, optimizer, device):
    # Set the model to training mode
    ext.eval() #dont train the extractor
    det.train() #train detector
    # Initialize the loss
    train_loss_class = []
    train_loss_regr = []

    err_losses = []
    dist_losses = []
    curv_losses = []
    bb_losses = []

    # Loop over the training batches
    for (img, input_data, regr_label, class_label) in tqdm(dataloader):
        # Move the input and target data to the selected device
        img, input_data, regr_label, class_label = img.to(device), input_data.to(device), regr_label.to(device), class_label.to(device)
        # Compute the features
        features = ext(img)
        #concatenate features and input_data
        input = torch.cat((features, input_data), dim=1)
        # Zero the gradients
        optimizer.zero_grad()
        # Compute the output
        output = det(input)

        #regression 22 values
        #classification: 3 states, 3 next states, 7 signs
        regr_out = output[:, :22-16+2]
        err_out = regr_out[:, :2]
        dist_out = regr_out[:, 2]
        curv_out = regr_out[:, 3]
        bb_out = regr_out[:, 4:8]

        err_label = regr_label[:, :2]
        dist_label = regr_label[:, 2]
        curv_label = regr_label[:, 3]
        bb_label = regr_label[:, 4:8]

        state_out = output[:, 22-16+2:25-16+2]
        next_out = output[:, 25-16+2:28-16+2]
        sign_out = output[:, 28-16+2:]
        
        state_label = class_label[:, :3]
        next_label = class_label[:, 3:6]
        sign_label = class_label[:, 6:]

        # Compute the losses
        # regr_loss = regr_loss_fn(regr_out, regr_label)

        err_loss = 50.0*regr_loss_fn(err_out, err_label)
        dist_loss = 0.1*regr_loss_fn(dist_out, dist_label)
        curv_loss = 0.1*regr_loss_fn(curv_out, curv_label)
        bb_loss = 1.0*regr_loss_fn(bb_out, bb_label)


        state_loss = 0.5*class_loss_fn(state_out, state_label)
        next_loss = 0.1*class_loss_fn(next_out, next_label)
        sign_loss = 5.0*class_loss_fn(sign_out, sign_label)
        loss = err_loss + dist_loss + curv_loss + bb_loss + state_loss + next_loss + sign_loss

        
        # Compute the gradients
        loss.backward()
        # Update the weights
        optimizer.step()
        #batch loss
        c_loss = (state_loss + next_loss + sign_loss).detach().cpu().numpy()
        train_loss_class.append(c_loss)
        err_losses.append(err_loss.detach().cpu().numpy())
        dist_losses.append(dist_loss.detach().cpu().numpy())
        curv_losses.append(curv_loss.detach().cpu().numpy())
        bb_losses.append(bb_loss.detach().cpu().numpy())
    # Return the average training loss
    train_loss_c = np.mean(train_loss_class)
    err_loss = np.mean(err_losses)
    dist_loss = np.mean(dist_losses)
    curv_loss = np.mean(curv_losses)
    bb_loss = np.mean(bb_losses)
    # print(f"Training loss: {train_loss}")
    return train_loss_c, err_loss, dist_loss, curv_loss, bb_loss

def get_avg_loss(ext, det, dataloader, class_loss_fn, regr_loss_fn, device):
    ext.eval()
    det.eval()
    class_losses = []
    regr_losses = []
    with torch.no_grad():
        for (img, input_data, regr_label, class_label) in tqdm(dataloader):
            # Move the input and target data to the selected device
            img, input_data, regr_label, class_label = img.to(device), input_data.to(device), regr_label.to(device), class_label.to(device)
            # Compute the features
            features = ext(img)
            #concatenate features and input_data
            input = torch.cat((features, input_data), dim=1)
            # Compute the output
            output = det(input)
            
            #regression 22 values
            #classification: 3 states, 3 next states, 7 signs
            regr_out = output[:, :22]
            state_out = output[:, 22:25]
            next_out = output[:, 25:28]
            sign_out = output[:, 28:]
            
            state_label = class_label[:, :3]
            next_label = class_label[:, 3:6]
            sign_label = class_label[:, 6:]

            # Compute the losses
            regr_loss = regr_loss_fn(regr_out, regr_label)
            state_loss = class_loss_fn(state_out, state_label)
            next_loss = class_loss_fn(next_out, next_label)
            sign_loss = class_loss_fn(sign_out, sign_label)
            class_loss = state_loss + next_loss + sign_loss

            class_losses.append(class_loss.detach().cpu().numpy())
            regr_losses.append(regr_loss.detach().cpu().numpy())
    # Return the accuracy and test loss
    class_loss = np.mean(class_losses)
    regr_loss = np.mean(regr_losses)
    return class_loss, regr_loss

In [ ]:
# #load models
# detector.load_state_dict(torch.load('detector.pt'))
# feature_extractor.load_state_dict(torch.load('feature_extractor.pt'))

#parameters
lr = 0.001
epochs = 3
optimizer = torch.optim.Adam(detector.parameters(), lr=lr)
regr_loss_fn = nn.MSELoss()
class_loss_fn = nn.CrossEntropyLoss()

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_loss_c, err_loss, dist_loss, curv_loss, bb_loss = train_epoch(feature_extractor, detector, train_dataloader, class_loss_fn, regr_loss_fn, optimizer, device)
    print(f"err_loss: {err_loss}")
    print(f"dist_loss: {dist_loss}")
    print(f"curv_loss: {curv_loss}")
    print(f"bb_loss: {bb_loss}")
    print(f"Classification loss: {train_loss_c}")
    torch.save(detector.state_dict(), 'models/detector.pt')
    torch.save(feature_extractor.state_dict(), 'models/feature_extractor.pt')

In [ ]:
#testing
test_dataset = CsvDataset(folder='test_imgs')
test_dataloader = DataLoader(test_dataset, batch_size=100, shuffle=True)

#get accuracy
train_class_loss, train_regr_loss = get_avg_loss(feature_extractor, detector, train_dataloader, class_loss_fn, regr_loss_fn, device)
test_class_loss, test_regr_loss = get_avg_loss(feature_extractor, detector, test_dataloader, class_loss_fn, regr_loss_fn, device)

print(f"Training classification loss: {train_class_loss}")
print(f"Training regression loss: {train_regr_loss}\n")
print(f"Testing classification loss: {test_class_loss}")
print(f"Testing regression loss: {test_regr_loss}")

In [ ]:
print(test_dataset.data[0][0].shape)

In [ ]:
detector.load_state_dict(torch.load('models/detector.pt'))
feature_extractor.load_state_dict(torch.load('models/feature_extractor.pt'))

# #save pytorch model
# torch.save(detector.state_dict(), 'detector.pt')
# torch.save(feature_extractor.state_dict(), 'feature_extractor.pt')

#save the model so that opencv can load it
import torch
import torch.onnx
import torchvision
import torchvision.models as models
import sys

device = torch.device('cpu')
detector.to(device)
feature_extractor.to(device)
 
onnx_detector_path = "models/detector.onnx"
onnx_feature_extractor_path = "models/feature_extractor.onnx"

# set the model to inference mode
detector.eval()
feature_extractor.eval()
 
# Create some sample input in the shape this model expects 
# This is needed because the convertion forward pass the network once 
dummy_input = torch.randn(1, 3, 240, 320)
dummy_input2 = torch.randn(1, 20484)
torch.onnx.export(feature_extractor, dummy_input, onnx_feature_extractor_path, verbose=True)
torch.onnx.export(detector, dummy_input2, onnx_detector_path, verbose=True)


In [ ]:
#test with opencv
sample_image = "training_imgs/img_1.png"
images = [cv.imread(f"training_imgs/img_{i+1}.png") for i in range(100)]
 
#The Magic:
det =  cv.dnn.readNetFromONNX(onnx_detector_path) 
ext = cv.dnn.readNetFromONNX(onnx_feature_extractor_path)

for i in tqdm(range(100)):
    image = images[i]
    blob = cv.dnn.blobFromImage(image, 1.0, (320, 240),(0, 0, 0), swapRB=True, crop=False)
    ext.setInput(blob)
    features = ext.forward()
    # print(features.shape)
    action_vec = np.ones((1,4))
    input = np.concatenate((features, action_vec), axis=1)
    # print(input.shape)
    det.setInput(input)
    preds = det.forward()

print(f"Predictions: {preds}")
print(f"Predictions shape: {preds.shape}")